- 2020-08-10
- OS: Windows 10

coco
- info
  - description
  - url
  - version
  - year
  - contributor
  - data_created
- licenses(list)
  - url
  - id
  - name
- images(list)
  - license
  - file_name
  - coco_url
  - height
  - width
  - date_captured
  - flickr_url
  - id
- annotations(list)
  - segmentation
  - area
  - iscrowd
  - image_id
  - bbox: \[xmin, ymin, width, height\]
  - category_id
  - id
- categories(list)
  - supercategory
  - id
  - name

In [1]:
instances_path = "instances_train2017.json"
names_path = "coco.names"
output_file_path = "train2017.txt"

In [2]:
import json
from collections import OrderedDict
from tqdm.notebook import tqdm

coco = json.load(open(instances_path))
images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]
replaced_name = {
    "couch": "sofa",
    "airplane": "aeroplane",
    "tv": "tvmonitor",
    "motorcycle": "motorbike",
}

class_to_id = {}
id_to_class = {}
with open(names_path, "r") as fd:
    index = 0
    for class_name in fd:
        class_name = class_name.strip()
        if len(class_name) != 0:
            id_to_class[index] = class_name
            class_to_id[class_name] = index
            index += 1

dataset = {}

for annotation in tqdm(annotations, desc="Parsing"):
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]

    file_name = None
    image_height = 0
    image_width = 0
    for image in images:
        if image["id"] == image_id:
            file_name = image["file_name"]
            image_height = image["height"]
            image_width = image["width"]
            break

    if file_name is None:
        continue

    class_id = None
    for category in categories:
        if category["id"] == category_id:
            category_name = category["name"]
            if category_name in replaced_name.keys():
                category_name = replaced_name[category_name]

            class_id = class_to_id.get(category_name)
            break

    if class_id is None:
        continue

    x_center = annotation["bbox"][0] + annotation["bbox"][2] / 2
    x_center /= image_width
    y_center = annotation["bbox"][1] + annotation["bbox"][3] / 2
    y_center /= image_height
    width = annotation["bbox"][2] / image_width
    height = annotation["bbox"][3] / image_height

    if dataset.get(image_id):
        dataset[image_id][1].append(
            [class_id, x_center, y_center, width, height]
        )
    else:
        dataset[image_id] = [
            file_name,
            [[class_id, x_center, y_center, width, height]],
        ]

dataset = OrderedDict(sorted(dataset.items()))

with open(output_file_path, "w") as fd:
    for image_id, bboxes in tqdm(dataset.items(), desc="Saving"):
        data = bboxes[0]
        for bbox in bboxes[1]:
            data += " "
            data += "{:d},".format(bbox[0])
            data += "{:8.6f},".format(bbox[1])
            data += "{:8.6f},".format(bbox[2])
            data += "{:8.6f},".format(bbox[3])
            data += "{:8.6f}".format(bbox[4])

        data += "\n"
        fd.write(data)